In [7]:
import os.path
import numpy as np
import pandas as pd
from time import time
from os import listdir
import glob
from os.path import isfile, join

In [8]:
def read_csv(filename):
    # read revise csv file and print cost time
    t0 = time()
    df = pd.read_csv(filename, error_bad_lines=False)
    print("time for read csv: %.2f" % (time()-t0))
    return df

In [9]:
def file_csv_list(target_path):
    return [f for f in listdir(target_path) if isfile(join(target_path, f))]

In [10]:
def save_dataframe(df, out_filename):
   # write to csv and no index
    t0 = time()
    df.to_csv(out_filename + ".csv", index=False, encoding='utf-8')
#     df.to_csv(out_filename + ".csv", encoding='utf-8')
    print("time for output csv file: %.2f" % (time()-t0))

In [11]:
def save_dataframe_with_id(df, out_filename):
   # write to csv and no index
    t0 = time()
    df.to_csv(out_filename + ".csv", encoding='utf-8')
#     df.to_csv(out_filename + ".csv", encoding='utf-8')
    print("time for output csv file: %.2f" % (time()-t0))

In [12]:
def paste_kid_to_df(df, filename):
    
    labels_ = np.load(filename)
    df['label'] = labels_
    df = df.fillna(0)
    return df

# batch write kid file

In [13]:
# set configure
# path = "../DATA_FULL/"
path = "./kid/"
out_path = path
for file in glob.glob(path + "*.csv"):
    keyword = "_".join(file.split("_")[-2:]) 
#     filter .csv
    keyword = keyword[:-4]
    if("kid" in keyword):
        print(keyword)
        npy_match_list = glob.glob(path + "*" + keyword + "*.npy")
        if(len(npy_match_list) == 1):
            print("find npy file" , npy_match_list[0])
            df = read_csv(file)
            df = paste_kid_to_df(df, npy_match_list[0])
            save_dataframe(df, out_path + keyword + "_with_label")
        elif(len(npy_match_list) == 0):
            print("cannot match %s to npy file" % file)
        if(len(npy_match_list) == 2):
            print("match %s more than one npy file skip..." % file)
        
   

kid_1
find npy file ./kid\label_K6_de_with_kid_1_06281.npy
time for read csv: 0.92
time for output csv file: 5.06
kid_2
find npy file ./kid\label_K5_de_with_kid_2_06281.npy
time for read csv: 0.43
time for output csv file: 2.39
kid_3
find npy file ./kid\label_K5_de_with_kid_3_06281.npy
time for read csv: 0.26
time for output csv file: 1.38
kid_4
find npy file ./kid\label_K5_de_with_kid_4_06281.npy
time for read csv: 0.30
time for output csv file: 1.52
kid_5
find npy file ./kid\label_K5_de_with_kid_5_06281.npy
time for read csv: 0.27
time for output csv file: 1.41
kid_6
find npy file ./kid\label_K5_de_with_kid_6_06281.npy
time for read csv: 0.35
time for output csv file: 1.93
kid_7
find npy file ./kid\label_K5_de_with_kid_7_06281.npy
time for read csv: 0.38
time for output csv file: 2.11
kid_8
find npy file ./kid\label_K5_de_with_kid_8_06281.npy
time for read csv: 1.17
time for output csv file: 6.32
kid_None
cannot match ./kid\table_with_kid_None.csv to npy file
kid_seldom
cannot match 

# LOAD MINING USER

In [204]:
# set configure
# path = "../DATA_FULL/"
path = "./"
filename = "DNA_KMEANS_RESULT_ID_NEW"
relative_filename = path + filename + ".csv"

In [205]:
user_df =  pd.read_csv(relative_filename, error_bad_lines=False)
# user_df = user_df[user_df["MINING_DW_SUBSCR_NO"] != "MINING_DW_SUBSCR_NO"]
# user_df["MINING_DW_SUBSCR_NO"] = user_df["MINING_DW_SUBSCR_NO"].apply(np.int64)

# check data is duplicate or not

In [206]:
user_df[user_df["MINING_DW_SUBSCR_NO"].duplicated(keep=False)]

,MINING_DW_SUBSCR_NO,DW_PARTY_ID,Groups


In [207]:
# set configure
# path = "../DATA_FULL/"
path = "./kid/"
filename = "kid_1_with_label"
relative_filename = path + filename + ".csv"
label_df =  pd.read_csv(relative_filename, error_bad_lines=False)
# df[df.duplicated(["MINING_DW_SUBSCR_NO"], keep=False)]


In [208]:
users_count = len(user_df)

# user_df.sort_values(["AGE"], inplace = True)
user_df = user_df.set_index("MINING_DW_SUBSCR_NO")
user_df.sort_index(inplace = True)

# user_df["Groups"] = np.nan
# user_df["label"] = np.nan
user_df["Groups"] = "None"
user_df["label"] = np.nan
# user_df["label"] = user_df["label"].apply(np.int64)


# paste all kid

In [209]:
# set configure
# path = "../DATA_FULL/"
path = "./kid/"
out_path = path
for file in glob.glob(path + "*.csv"):
    if("with_label" in file):
        # get Group id
        group_num  = file.split("_")[1].strip()
        print("filename = %s , Groups = %s" % (file, group_num))
        wants_cols = ["MINING_DW_SUBSCR_NO", "label"]
        label_df = pd.read_csv(file, error_bad_lines=False, usecols = wants_cols)
        print("group_num =", group_num)
        label_df["Groups"] = group_num
        print("len(label_df[Groups]) =", len(label_df["Groups"]))

#
        label_df = label_df.set_index("MINING_DW_SUBSCR_NO")
        label_df.sort_index(inplace = True)
#         label_df = label_df[label_df['MINING_DW_SUBSCR_NO'].isin(user_df['MINING_DW_SUBSCR_NO'])]
#         label_df = label_df[label_df.index.isin(user_df.index)]
        new_user_label = user_df[['Groups', 'label']]
        new_user_label.update(label_df)
        user_df[['Groups', 'label']] = new_user_label


#         user_df["label"].update(label_df["label"], raise_conflict=True)
#         user_df.update(label_df)


#         break

filename = ./kid\kid_1_with_label.csv , Groups = 1
group_num = 1
len(label_df[Groups]) = 407597


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3863: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  raise_on_error=True)


filename = ./kid\kid_2_with_label.csv , Groups = 2
group_num = 2
len(label_df[Groups]) = 184644
filename = ./kid\kid_3_with_label.csv , Groups = 3
group_num = 3
len(label_df[Groups]) = 105809
filename = ./kid\kid_4_with_label.csv , Groups = 4
group_num = 4
len(label_df[Groups]) = 116958
filename = ./kid\kid_5_with_label.csv , Groups = 5
group_num = 5
len(label_df[Groups]) = 110474
filename = ./kid\kid_6_with_label.csv , Groups = 6
group_num = 6
len(label_df[Groups]) = 146047
filename = ./kid\kid_7_with_label.csv , Groups = 7
group_num = 7
len(label_df[Groups]) = 164149
filename = ./kid\kid_8_with_label.csv , Groups = 8
group_num = 8
len(label_df[Groups]) = 490166


In [210]:
print(len(user_df))
print(len(user_df[user_df["Groups"] == "4"]))
print(len(user_df[user_df["label"] == 1]))

2596797
116958
400403


In [211]:
user_df.head()

,DW_PARTY_ID,Groups,label
MINING_DW_SUBSCR_NO,,,
1179,648851,None,NaN
1181,3688656,2,1.0
11121,6913560,3,3.0
11145,1849166,None,NaN
11149,2143241,2,0.0


In [212]:
user_df.dtypes

DW_PARTY_ID      int64
Groups          object
label          float64
dtype: object

In [213]:
user_df["label"] = user_df["label"].fillna(-1)
user_df["label"] = user_df["label"].apply(np.int64)

In [214]:
user_df.head()

,DW_PARTY_ID,Groups,label
MINING_DW_SUBSCR_NO,,,
1179,648851,None,-1
1181,3688656,2,1
11121,6913560,3,3
11145,1849166,None,-1
11149,2143241,2,0


In [215]:
save_dataframe_with_id(user_df, out_path +  "single_id_age_group_label")

time for output csv file: 5.56


In [ ]:
print("origin users_count", users_count)
print("current users_count", len(user_df))

In [8]:
wants_cols = ["MINING_DW_SUBSCR_NO"]
df =  pd.read_csv(relative_filename, usecols = wants_cols, error_bad_lines=False)

In [9]:
df.head()

,MINING_DW_SUBSCR_NO
0,10000000027
1,10000000075
2,10000000084
3,10000000123
4,10000000164


In [219]:
path = "./kid/"
filename = "label_K6_de_with_kid_1_06281"
FILE = path + filename + ".npy"
labels_ = np.load(FILE)
filename = "table_with_kid_1"
relative_filename = path + filename + ".csv"
df = read_csv(relative_filename)
df['label'] = labels_
print(len(df[df['label'] == 5]))
df = df.fillna(0)
out_path = "./CDR_FINAL/"


time for read csv: 0.94
32089


In [11]:
df.head()

,MINING_DW_SUBSCR_NO,label
0,10000000027,2
1,10000000075,0
2,10000000084,4
3,10000000123,3
4,10000000164,0


time for output csv file: 0.22
